In [2]:
import os 
import mlflow as mf 
import joblib
import torch
import transformers
import numpy as np
from tqdm import tqdm

from torch.utils.data import DataLoader
import ftzard.utils.mlflow as mf_utils


from hydra import initialize, compose
from warnings import filterwarnings

from transformers import DataCollatorWithPadding, AutoTokenizer
import dagstermill as dgm
from ftzard.utils.common import get_current_date_time

from sklearn.metrics import classification_report, accuracy_score, f1_score


filterwarnings("ignore")

In [2]:
base_path = '/app/ftzard'
config_path = f'{base_path}/config/'
try:
    os.symlink(config_path, "config_link")
except Exception as e:
    print("Symlink already created...")
config_name = 'config'

Symlink already created...


In [3]:
with initialize(version_base=None, config_path="config_link"):
    cfg = compose(config_name=config_name)
    tracking_uri, experiment_name = cfg.MLFLOW.TRACKING.URI, cfg.MLFLOW.EXPERIMENT.NAME
    mlflow_model_name = cfg.MLFLOW.MODEL.NAME 
    

In [4]:
os.environ['MLFLOW_TRACKING_URI'] = tracking_uri
run_name = get_current_date_time()
base_run_name = "INFERENCE"
alias = 'champion'
print("Base Run Name:", base_run_name)
print('Mlflow Experiment Name: ', experiment_name)
print('Mlflow Run Name: ', run_name)
print('Mlflow Model Name: ', mlflow_model_name)
print("Mlflow Model Alias: ", alias)

Base Run Name: INFERENCE
Mlflow Experiment Name:  senetiment_analysis
Mlflow Run Name:  2024-07-02_11:54
Mlflow Model Name:  FalconSentiAnalysis
Mlflow Model Alias:  champion


In [5]:
datasets = joblib.load(f"{base_path}/data/tokenized_dataset.joblib")

INFO:datasets:PyTorch version 2.0.1 available.


In [6]:
datasets = datasets["datasets"]

In [7]:
print(datasets['test'])

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 480
})


In [8]:
## Load Model ##
model_uri = f"models:/{mlflow_model_name}@{alias}"
components = mf.transformers.load_model(model_uri, return_type="components")

INFO:alembic.runtime.migration:Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.
2024/07/02 11:54:23 INFO mlflow.transformers: 'models:/FalconSentiAnalysis@champion' resolved as '/app/ftzard/pipeline/notebooks/mlruns/1/9953da46234843f1a475e042a89a9b2c/artifacts'
Loading checkpoint shards: 100%|████████████████████████████████████████████| 2/2 [00:13<00:00,  6.99s/it]
Some weights of FalconForSequenceClassification were not initialized from the model checkpoint at tiiuae/falcon-7b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
tokenizer = components["tokenizer"]
model = components["model"]
del components
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

In [10]:
print(model)

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): FalconForSequenceClassification(
      (transformer): FalconModel(
        (word_embeddings): Embedding(65024, 4544)
        (h): ModuleList(
          (0-31): 32 x FalconDecoderLayer(
            (self_attention): FalconAttention(
              (rotary_emb): FalconRotaryEmbedding()
              (query_key_value): lora.Linear(
                (base_layer): FalconLinear(in_features=4544, out_features=4672, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05187051351110231, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4544, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=4672, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B):

In [11]:
experiment_id = mf_utils.create_experiment(exp_name=experiment_name)

The provided experiment name senetiment_analysis already exists, the run will be logged in this experiment.
                                 


In [17]:

'''
Preparing Data For Evaluation
'''
eval_dataloader = DataLoader(
    datasets["test"],
    batch_size=32,
    shuffle=False,
    collate_fn=DataCollatorWithPadding(tokenizer=tokenizer),
)

device = model.device

print("Device of PEFT MODEL: ", device)

def get_predictions(batch):
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    # token_type_ids = batch["token_type_ids"].to(device)

    with torch.no_grad():
        outputs = model(**{"input_ids":input_ids, "attention_mask":attention_mask})
        logits = outputs.logits
        predicted_labels = torch.argmax(logits, dim=-1) # Move predicted_labels to the same device

    return predicted_labels.cpu().numpy(), logits

print('Getting predictions from best model....')


base_run_id = mf_utils.get_run_id_by_name(run_name=base_run_name, 
                                             experiment_ids=[experiment_id])
with mf.start_run(run_id=base_run_id, experiment_id=experiment_id):
    run_id = mf_utils.get_run_id_by_name(run_name=run_name, 
                                             experiment_ids=[experiment_id], nested=True)
    mf.start_run(run_id=run_id, run_name=run_name, experiment_id=experiment_id,nested=True)
    all_predictions, all_logits = [], []
    for batch in tqdm(eval_dataloader):
        predictions, logits = get_predictions(batch)
        all_predictions.extend(predictions)
        all_logits.extend(logits)
    try:
        mf.log_param("NumberOfReocrds", len(all_logits))
    except Exception as e:
        print(e)
    mf.end_run()

Device of PEFT MODEL:  cuda:0
Getting predictions from best model....


100%|█████████████████████████████████████████████████████████████████████| 15/15 [00:06<00:00,  2.39it/s]


## ONLY WHEN YOU HAVE A LABELLED TEST SET (OPTIONAL)

In [19]:
# print('Fetching true labels from dataset....')
# true = []
# for batch in tqdm(eval_dataloader):
#     true.extend(batch['labels'])

# print('                  Classification Report         ')
# print("-----------------------------------------------------")
# print(classification_report(true, all_predictions))

Fetching true labels from dataset....


100%|████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 278.44it/s]

                  Classification Report         
-----------------------------------------------------
              precision    recall  f1-score   support

           0       0.89      0.82      0.85       240
           1       0.83      0.90      0.86       240

    accuracy                           0.86       480
   macro avg       0.86      0.86      0.86       480
weighted avg       0.86      0.86      0.86       480



In [20]:
# with mf.start_run(run_id=base_run_id, run_name=base_run_name, experiment_id=experiment_id):
#     with mf.start_run(run_id=run_id, run_name=run_name, experiment_id=experiment_id,
#                      nested=True):
#         mf.log_metric("accuracy", accuracy_score(true, all_predictions))
#         mf.log_metric("f1_score", accuracy_score(true, all_predictions))

In [21]:
outputs = {"data": datasets["test"],
           "predicted_labels": all_predictions,
          "logits": [i.cpu().numpy() for i in all_logits]}

In [22]:
# save_path = f"{base_path}/data/predictions.joblib"
# with open(save_path, 'wb') as f:
#     joblib.dump(outputs, f)
metadata = {"run_name": run_name,
           "run_id":run_id,
           "base_run_id": base_run_id,
           "base_run_name": base_run_name}

In [23]:
dgm.yield_result(outputs, output_name="predictions_logits")
dgm.yield_result(metadata, output_name='step5_run_metadata')

{'run_name': '2024-07-02_11:54',
 'run_id': 'c02f01049b5a4781b364917430552dc7',
 'base_run_id': 'a88ae9eaf69a4ec283168c1507a5bc0b',
 'base_run_name': 'INFERENCE'}